In [11]:
import pandas as pd

In [12]:
news_data=pd.read_csv('/kaggle/input/appldata/news2.csv')

In [14]:
import pandas as pd
import spacy
from nltk.stem import PorterStemmer
from textblob import TextBlob
import nltk
from transformers import pipeline
from nltk.corpus import stopwords

nltk.download('stopwords')

nlp = spacy.load("en_core_web_sm")

stemmer = PorterStemmer()

sentiment_analyzer = pipeline("sentiment-analysis")


df = pd.DataFrame(news)

def preprocess_text(text):
    doc = nlp(text)
    lemmatized_text = " ".join([token.lemma_ for token in doc if token.text.lower() not in stopwords.words("english")])
    
    stemmed_text = " ".join([stemmer.stem(word) for word in text.split() if word.lower() not in stopwords.words("english")])
    
    return lemmatized_text, stemmed_text

def get_textblob_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity, blob.sentiment.subjectivity

def get_transformers_sentiment(text):
    result = sentiment_analyzer(text)[0]
    return result['label'], result['score']

lemmatized_texts = []
stemmed_texts = []
textblob_polarities = []
textblob_subjectivities = []
transformers_labels = []
transformers_scores = []

for summary in df['summary']:
    lemmatized, stemmed = preprocess_text(summary)
    lemmatized_texts.append(lemmatized)
    stemmed_texts.append(stemmed)
    
    polarity, subjectivity = get_textblob_sentiment(summary)
    textblob_polarities.append(polarity)
    textblob_subjectivities.append(subjectivity)
    
    label, score = get_transformers_sentiment(summary)
    transformers_labels.append(label)
    transformers_scores.append(score)

df['lemmatized_summary'] = lemmatized_texts
df['stemmed_summary'] = stemmed_texts
df['textblob_polarity'] = textblob_polarities
df['textblob_subjectivity'] = textblob_subjectivities
df['transformers_sentiment_label'] = transformers_labels
df['transformers_sentiment_score'] = transformers_scores

print(df[['title', 'summary', 'lemmatized_summary', 'stemmed_summary', 'textblob_polarity', 'textblob_subjectivity', 'transformers_sentiment_label', 'transformers_sentiment_score']])


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


                                                title  \
0   Taiwan Semi Founder Calls For New Strategy Aft...   
1   A Tech Renaissance Could Offer Enticing Opport...   
2   Is Nvidia the Top Artificial Intelligence  ( A...   
3   Computer Hardware Manufacturing Market Set to ...   
4   Should Vanguard Russell 1000 Growth ETF  ( VON...   
5   Should You Invest in the Fidelity MSCI Informa...   
6   Is Elon Musk Apple TV's 'Slow Horses' Fan? His...   
7   Tesla Tops Nvidia And Apple In Innovation: Ana...   
8   Apple Watch Ultra Could Feature Satellite Text...   
9   HPQ Soars 21% YTD: Should You Buy, Sell or Hol...   
10  Copa América, Summer Olympics, Donald Trump, P...   
11  Competitor Analysis: Evaluating Apple And Comp...   
12  Taiwan Semi November Revenue Ups 34% Strengthe...   
13  Apple Inc.  ( AAPL )  is Attracting Investor A...   
14  Want to Buy Nvidia Stock Before the End of the...   
15               Will the Stock Market Crash in 2025?   
16  1 Warren Buffett Stock That

In [16]:
news_df['time_published'] = pd.to_datetime(news_df['time_published'])


In [19]:
news_df.columns

Index(['title', 'url', 'time_published', 'authors', 'summary', 'banner_image',
       'source', 'category_within_source', 'source_domain', 'topics',
       'overall_sentiment_score', 'overall_sentiment_label',
       'ticker_sentiment', 'lemmatized_summary', 'stemmed_summary',
       'textblob_polarity', 'textblob_subjectivity',
       'transformers_sentiment_label', 'transformers_sentiment_score'],
      dtype='object')

In [20]:
news_df.to_csv('sentiment_data.csv', columns=[
    'time_published','overall_sentiment_score', 'overall_sentiment_label',
       'ticker_sentiment', 'lemmatized_summary', 'stemmed_summary',
       'textblob_polarity', 'textblob_subjectivity',
       'transformers_sentiment_label', 'transformers_sentiment_score'], index=False)
